In [1]:
# Run this to install all the necessary packages

!pip install numpy torch scikit-learn requests matplotlib

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Import all the required packages

import os
import torch
import json
import requests
import numpy as np
import random
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

In [3]:
# Prepare dataset for training

def prepare_fibonacci_dataset(sequence_length=20, total_numbers=100000):
    # Generate Fibonacci sequence mod 10
    fib_mod10 = [0, 1]
    for _ in range(total_numbers - 2):
        fib_mod10.append((fib_mod10[-1] + fib_mod10[-2]) % 10)
    
    # Create sequences and corresponding labels
    X, y = [], []
    for i in range(len(fib_mod10) - sequence_length):
        X.append(fib_mod10[i : i + sequence_length])
        y.append(fib_mod10[i + sequence_length])

    # Convert to numpy arrays
    X = np.array(X) / 9.0  # Normalize input (0 to 1)
    y = np.array(y)  # Target is a digit (0-9)

    # Convert to PyTorch tensors
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.long)

    # Split dataset
    X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

    print("✅ Fibonacci dataset prepared and stored in memory!")
    return X_train, X_test, y_train, y_test

# Call the function and store the dataset in memory
X_train, X_test, y_train, y_test = prepare_fibonacci_dataset()

✅ Fibonacci dataset prepared and stored in memory!


In [4]:
# Define LSTM Model 

class FibonacciLSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, output_size=10):
        super(FibonacciLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x.unsqueeze(-1), (h0, c0))
        out = self.fc(out[:, -1, :])  # Get output from last time step

        return out

In [6]:
# Train the model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FibonacciLSTM().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create DataLoader
batch_size = 64
train_loader_fib = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)

# Train model (Tracking confidence and accuracy)
num_epochs = 10
train_losses = []
confidence_scores_fib = []
accuracies_fib = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    epoch_confidence = []
    correct_predictions = 0
    total_samples = 0

    for X_batch, y_batch in train_loader_fib:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)

        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # **Compute Confidence Score**
        probabilities = torch.softmax(outputs, dim=1)  # Convert logits to probabilities
        max_confidence, predicted = torch.max(probabilities, dim=1)  # Get max confidence score
        epoch_confidence.extend(max_confidence.detach().cpu().numpy())

        # **Compute Accuracy**
        correct_predictions += (predicted == y_batch).sum().item()
        total_samples += y_batch.size(0)

    avg_loss = total_loss / len(train_loader_fib)
    train_losses.append(avg_loss)

    # Store average confidence & accuracy per epoch
    confidence_scores_fib.append(np.mean(epoch_confidence))
    accuracies_fib.append((correct_predictions / total_samples) * 100)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Confidence: {confidence_scores_fib[-1]:.2f}, Accuracy: {accuracies_fib[-1]:.2f}%")

print("✅ Fibonacci Model trained and stored in memory!")

# **Save Fibonacci model results**
results_fib = {
    "confidence_scores": [float(score) for score in confidence_scores_fib],
    "accuracies": [float(acc) for acc in accuracies_fib] 
}

with open("fib_results.json", "w") as f:
    json.dump(results_fib, f)

print("✅ Fibonacci model results saved successfully!")

Epoch [1/10], Loss: 0.7621, Confidence: 0.66, Accuracy: 74.08%
Epoch [2/10], Loss: 0.0065, Confidence: 0.99, Accuracy: 100.00%
Epoch [3/10], Loss: 0.0017, Confidence: 1.00, Accuracy: 100.00%
Epoch [4/10], Loss: 0.0007, Confidence: 1.00, Accuracy: 100.00%
Epoch [5/10], Loss: 0.0003, Confidence: 1.00, Accuracy: 100.00%
Epoch [6/10], Loss: 0.0001, Confidence: 1.00, Accuracy: 100.00%
Epoch [7/10], Loss: 0.0001, Confidence: 1.00, Accuracy: 100.00%
Epoch [8/10], Loss: 0.0000, Confidence: 1.00, Accuracy: 100.00%
Epoch [9/10], Loss: 0.0000, Confidence: 1.00, Accuracy: 100.00%
Epoch [10/10], Loss: 0.0000, Confidence: 1.00, Accuracy: 100.00%
✅ Fibonacci Model trained and stored in memory!
✅ Fibonacci model results saved successfully!


In [7]:
# Evaluate model performance

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for i in range(len(X_test)):
        X = X_test[i].unsqueeze(0).to(device)
        y = y_test[i].to(device)

        outputs = model(X)
        _, predicted = torch.max(outputs, 1)

        correct += (predicted == y).sum().item()
        total += 1

accuracy = correct / total
print(f"📊 Fibonacci Test Accuracy: {accuracy * 100:.2f}%")

📊 Fibonacci Test Accuracy: 100.00%


In [9]:
# Compare model performance with random guessing

random_accuracy = sum(random.randint(0, 9) == y.item() for y in y_test) / len(y_test)
print(f"🎲 Random Guessing Accuracy: {random_accuracy * 100:.2f}%")

🎲 Random Guessing Accuracy: 9.98%


In [10]:
# Predict the next digit given the first 20 digits (This is for you to try by giving input)

def predict_next_fibonacci(input_digits):
    # Remove non-digit characters
    clean_digits = [d for d in input_digits if d.isdigit()]
    
    # Convert to numerical format
    digits = np.array([int(d) for d in clean_digits])

    # Check if input length is valid
    sequence_length = 20
    if len(digits) < sequence_length:
        print(f"⚠️ Input must be at least {sequence_length} digits long!")
        return
    
    # Use the last `sequence_length` digits
    digits = digits[-sequence_length:]

    # Normalize input (scale between 0 and 1)
    X_input = np.array(digits) / 9.0
    X_tensor = torch.tensor(X_input, dtype=torch.float32).unsqueeze(0).to(device)

    # Predict the next digit
    with torch.no_grad():
        output = model(X_tensor)
        predicted_digit = torch.argmax(output, dim=1).item()

    print(f"🔢 Predicted next Fibonacci digit after {input_digits} is: {predicted_digit}")

# Get user input and predict
user_input = input("Enter a sequence of Fibonacci Mod 10 digits: ")
predict_next_fibonacci(user_input)

Enter a sequence of Fibonacci Mod 10 digits:  011


⚠️ Input must be at least 20 digits long!
